In [ ]:
# Основное задание + пункты 8,9
WHITE = 1
BLACK = 2


# Удобная функция для вычисления цвета противника
def opponent(color):
    if color == WHITE:
        return BLACK
    else:
        return WHITE


def print_board(board):  # Доска в текстовом виде
    print('     +----+----+----+----+----+----+----+----+')
    for row in range(7, -1, -1):
        print(' ', row, end='  ')
        for col in range(8):
            print('|', board.cell(row, col), end=' ')
        print('|')
        print('     +----+----+----+----+----+----+----+----+')
    print(end='        ')
    for col in range(8):
        print(col, end='    ')
    print()


def main():
    # Создаём шахматную доску
    board = Board()
    # Цикл ввода команд игроков
    while True:
        # Выводим положение фигур на доске
        print_board(board)
        # Подсказка по командам
        print('Команды:')
        print('    exit                               -- выход')
        print('    move <row> <col> <row1> <col1>     -- ход из клетки (row, col)')
        print('                                          в клетку (row1, col1)')
        # Выводим приглашение игроку нужного цвета
        if board.current_player_color() == WHITE:
            print('Ход белых:')
        else:
            print('Ход чёрных:')
        command = input()
        if command == 'exit':
            break
        move_type, row, col, row1, col1 = command.split()
        row, col, row1, col1 = int(row), int(col), int(row1), int(col1)
        if board.move_piece(row, col, row1, col1):
            print('Ход успешен')
        else:
            print('Координаты некорректы! Попробуйте другой ход!')


def correct_coords(row, col):
    '''Функция проверяет, что координаты (row, col) лежат внутри доски'''
    return 0 <= row < 8 and 0 <= col < 8


class Board:
    def __init__(self):
        self.color = WHITE
        self.field = []
        for row in range(8):
            self.field.append([None] * 8)
        self.field[0] = [
            Rook(WHITE), Knight(WHITE), Bishop(WHITE), Queen(WHITE),
            King(WHITE), Bishop(WHITE), Knight(WHITE), Rook(WHITE)
        ]
        self.field[1] = [
            Pawn(WHITE), Pawn(WHITE), Pawn(WHITE), Pawn(WHITE),
            Pawn(WHITE), Pawn(WHITE), Pawn(WHITE), Pawn(WHITE)
        ]
        self.field[6] = [
            Pawn(BLACK), Pawn(BLACK), Pawn(BLACK), Pawn(BLACK),
            Pawn(BLACK), Pawn(BLACK), Pawn(BLACK), Pawn(BLACK)
        ]
        self.field[7] = [
            Rook(BLACK), Knight(BLACK), Bishop(BLACK), Queen(BLACK),
            King(BLACK), Bishop(BLACK), Knight(BLACK), Rook(BLACK)
        ]

    def current_player_color(self):
        return self.color

    def cell(self, row, col):
        '''Возвращает строку из двух символов. Если в клетке (row, col)
        находится фигура, символы цвета и фигуры. Если клетка пуста,
        то два пробела.'''
        piece = self.field[row][col]
        if piece is None:
            return '  '
        color = piece.get_color()
        c = 'w' if color == WHITE else 'b'
        return c + piece.char()

    def get_piece(self, row, col):
        if correct_coords(row, col):
            return self.field[row][col]
        else:
            return None

    def move_piece(self, row, col, row1, col1):
        '''Переместить фигуру из точки (row, col) в точку (row1, col1).
        Если перемещение возможно, метод выполнит его и вернёт True.
        Если нет --- вернёт False'''

        if not correct_coords(row, col) or not correct_coords(row1, col1):
            return False
        if row == row1 and col == col1:
            return False  # нельзя пойти в ту же клетку
        piece = self.field[row][col]
        if piece is None:
            return False
        if piece.get_color() != self.color:
            return False
        if self.field[row1][col1] is None:
            if not piece.can_move(self, row, col, row1, col1):
                return False
        elif self.field[row1][col1].get_color() == opponent(piece.get_color()):
            if not piece.can_attack(self, row, col, row1, col1):
                return False
        else:
            return False
        if self.field[row][col].char() == 'K':
            self.field[row][col].castling = False
        elif self.field[row][col].char() == 'R':
            self.field[row][col].castling = False
        self.field[row][col] = None  # Снять фигуру.
        self.field[row1][col1] = piece  # Поставить на новое место.
        self.color = opponent(self.color)
        return True

    def move_and_promote_pawn(self, row, col, row1, col1, char):
        if self.field[row][col].char() == 'P':
            if self.move_piece(row, col, row1, col1):
                if char == 'N':
                    self.field[row1][col1] = Knight(self.field[row1][col1].get_color())
                    return True
                if char == 'Q':
                    self.field[row1][col1] = Queen(self.field[row1][col1].get_color())
                    return True
                if char == 'R':
                    self.field[row1][col1] = Rook(self.field[row1][col1].get_color())
                    return True
                if char == 'B':
                    self.field[row1][col1] = Bishop(self.field[row1][col1].get_color())
                    return True
        return False

    def castling0(self):#Рокировка
        if self.color == WHITE:
            if (self.field[0][0] is not None) and (self.field[0][4] is not None):
                if self.field[0][4].char() == 'K':
                    if self.field[0][4].get_color() == WHITE and self.field[0][0].char() == 'R':
                        if self.field[0][0].get_color() == WHITE and \
                                self.field[0][1:4] == [None, None, None] and \
                                self.field[0][0].can_castling() and \
                                self.field[0][4].can_castling():
                            self.field[0][2] = self.field[0][4]
                            self.field[0][4] = None
                            self.field[0][3] = self.field[0][0]
                            self.field[0][0] = None
                            self.color = opponent(self.color)
                            return True
            return False
        else:
            if (self.field[7][0] is not None) and (self.field[7][4] is not None):
                if self.field[7][4].char() == 'K':
                    if self.field[7][4].get_color() == BLACK and self.field[7][0].char() == 'R':
                        if self.field[7][0].get_color() == BLACK and \
                                self.field[7][1:4] == [None, None, None] and \
                                self.field[7][0].can_castling() and \
                                self.field[7][4].can_castling():
                            self.field[7][2] = self.field[7][4]
                            self.field[7][4] = None
                            self.field[7][3] = self.field[7][0]
                            self.field[7][0] = None
                            self.color = opponent(self.color)
                            return True
            return False

    def castling7(self):
        if self.color == WHITE:
            if (self.field[0][7] is not None) and (self.field[0][4] is not None):
                if self.field[0][4].char() == 'K':
                    if self.field[0][4].get_color() == WHITE and self.field[0][7].char() == 'R':
                        if self.field[0][7].get_color() == WHITE and \
                                self.field[0][5:7] == [None, None] and \
                                self.field[0][7].can_castling() and \
                                self.field[0][4].can_castling():
                            self.field[0][6] = self.field[0][4]
                            self.field[0][4] = None
                            self.field[0][5] = self.field[0][7]
                            self.field[0][7] = None
                            self.color = opponent(self.color)
                            return True
            return False
        else:
            if (self.field[7][7] is not None) and (self.field[7][4] is not None):
                if self.field[7][4].char() == 'K':
                    if self.field[7][4].get_color() == BLACK and self.field[7][7].char() == 'R':
                        if self.field[7][7].get_color() == BLACK and \
                                self.field[7][5:7] == [None, None] and \
                                self.field[7][7].can_castling() and \
                                self.field[7][4].can_castling():
                            self.field[7][6] = self.field[7][4]
                            self.field[7][4] = None
                            self.field[7][5] = self.field[7][7]
                            self.field[7][7] = None
                            self.color = opponent(self.color)
                            return True
            return False


class Rook:

    def __init__(self, color):
        self.color = color
        self.castling = True

    def get_color(self):
        return self.color

    def char(self):
        return 'R'

    def can_move(self, board, row, col, row1, col1):
        # Невозможно сделать ход в клетку, которая не лежит в том же ряду
        # или столбце клеток.
        if row != row1 and col != col1:
            return False

        step = 1 if (row1 >= row) else -1
        for r in range(row + step, row1, step):
            # Если на пути по горизонтали есть фигура
            if not (board.get_piece(r, col) is None):
                return False

        step = 1 if (col1 >= col) else -1
        for c in range(col + step, col1, step):
            # Если на пути по вертикали есть фигура
            if not (board.get_piece(row, c) is None):
                return False

        return True

    def can_attack(self, board, row, col, row1, col1):
        return self.can_move(board, row, col, row1, col1)

    def can_castling(self):
        return self.castling


class Pawn:

    def __init__(self, color):
        self.color = color

    def get_color(self):
        return self.color

    def char(self):
        return 'P'

    def can_move(self, board, row, col, row1, col1):
        # Пешка может ходить только по вертикали
        # "взятие на проходе" не реализовано
        if col != col1:
            return False

        # Пешка может сделать из начального положения ход на 2 клетки
        # вперёд, поэтому поместим индекс начального ряда в start_row.
        if self.color == WHITE:
            direction = 1
            start_row = 1
        else:
            direction = -1
            start_row = 6

        # ход на 1 клетку
        if row + direction == row1:
            return True

        # ход на 2 клетки из начального положения
        if (row == start_row
                and row + 2 * direction == row1
                and board.field[row + direction][col] is None):
            return True

        return False

    def can_attack(self, board, row, col, row1, col1):
        direction = 1 if (self.color == WHITE) else -1
        return (row + direction == row1
                and (col + 1 == col1 or col - 1 == col1))


class Knight:
    '''Класс коня.'''

    def __init__(self, color):
        self.color = color

    def get_color(self):
        return self.color

    def char(self):
        return 'N'  

    def can_move(self, board, row, col, row1, col1):
        if not (0 <= row1 < 8 and 0 <= col1 < 8):
            return False
        if row == row1 and col == col1:
            return False
        if not board.get_piece(row1, col1) is None:
            if board.get_piece(row1, col1).get_color() == board.get_piece(row, col).get_color():
                return False
        if abs(col - col1) * abs(row - row1) == 2 and row != row1 and \
                col != col1:
            return True
        return False

    def can_attack(self, board, row, col, row1, col1):
        return self.can_move(self, board, row, col, row1, col1)


class King:
    '''Класс короля. Пока что заглушка, которая может ходить в любую клетку.'''

    def __init__(self, color):
        self.color = color
        self.castling = True

    def get_color(self):
        return self.color

    def char(self):
        return 'K'

    def can_move(self, board, row, col, row1, col1):  # король не может ходить сквозь фигуры и выходить за поле
        if not (0 <= row1 < 8 and 0 <= col1 < 8):
            return False
        if row == row1 and col == col1:
            return False
        if not board.get_piece(row1, col1) is None:
            if board.get_piece(row1, col1).get_color() == board.get_piece(row, col).get_color():
                return False

    def can_attack(self, board, row, col, row1, col1):
        return self.can_move(self, board, row, col, row1, col1)

    def can_castling(self):
        return self.castling

class Queen:
    def __init__(self, color):
        self.color = color

    def get_color(self):
        return self.color

    def char(self):
        return 'Q'

    def can_move(self, board, row, col, row1, col1):
        if not (0 <= row1 < 8 and 0 <= col1 < 8):
            return False
        if row == row1 and col == col1:
            return False
        if not board.get_piece(row1, col1) is None:
            if board.get_piece(row1, col1).get_color() == board.get_piece(row, col).get_color():
                return False
        if abs(row1 - row) == abs(col1 - col) or row == row1 or col == col1:
            if row == row1:
                if col1 >= col:
                    step = 1
                else:
                    step = -1
                for c in range(col + step, col1, step):
                    if not (board.get_piece(row, c) is None):
                        return False
            else:
                if row1 >= row:
                    step = 1
                else:
                    step = -1
                for r in range(row + step, row1, step):
                    if row - row1 == col - col1:
                        if not (board.get_piece(r, r - row + col) is None):
                            return False
                    if row - row1 == -col + col1:
                        if not (board.get_piece(r, -r + row + col) is None):
                            return False
                    if col == col1:
                        if not (board.get_piece(r, col) is None):
                            return False
            return True
        return False

    def can_attack(self, board, row, col, row1, col1):
        return self.can_move(self, board, row, col, row1, col1)


class Bishop:
    '''Класс слона.'''

    def __init__(self, color):
        self.color = color

    def get_color(self):
        return self.color

    def char(self):
        return 'B'

    def can_move(self, board, row, col, row1, col1):
        if not (0 <= row1 < 8 and 0 <= col1 < 8):
            return False
        if row == row1 and col == col1:
            return False
        if not board.get_piece(row1, col1) is None:
            if board.get_piece(row1, col1).get_color() == board.get_piece(row, col).get_color():
                return False
        if abs(row1 - row) == abs(col1 - col):
            if row == row1:
                if col1 >= col:
                    step = 1
                else:
                    step = -1
                for c in range(col + step, col1, step):
                    if not (board.get_piece(row, c) is None):
                        return False
            else:
                if row1 >= row:
                    step = 1
                else:
                    step = -1
                for r in range(row + step, row1, step):
                    if row - row1 == col - col1:
                        if not (board.get_piece(r, r - row + col) is None):
                            return False
                    if row - row1 == -col + col1:
                        if not (board.get_piece(r, -r + row + col) is None):
                            return False
                    if col == col1:
                        if not (board.get_piece(r, col) is None):
                            return False
            return True
        return False

    def can_attack(self, board, row, col, row1, col1):
        return self.can_move(self, board, row, col, row1, col1)


# __name__ -- специальная переменная, в которую python записывает имя
# файла (без .py), если этот файл импортирован из другого, и "__main__", если
# этот файл запущен как программа.
# Другими словами, следующие две строчки:
#   запустят функцию main, если файл запущен как программа;
#   не сделают ничего, если этот файл импортирован из другого.
# Второй случай реализуется, например, когда тестирующий скрипт импортирует
# классы из скрипта. В этом случае функция main не будет работать
# и портить вывод теста.


if __name__ == "__main__":
    main()

     +----+----+----+----+----+----+----+----+
  7  | bR | bN | bB | bQ | bK | bB | bN | bR |
     +----+----+----+----+----+----+----+----+
  6  | bP | bP | bP | bP | bP | bP | bP | bP |
     +----+----+----+----+----+----+----+----+
  5  |    |    |    |    |    |    |    |    |
     +----+----+----+----+----+----+----+----+
  4  |    |    |    |    |    |    |    |    |
     +----+----+----+----+----+----+----+----+
  3  |    |    |    |    |    |    |    |    |
     +----+----+----+----+----+----+----+----+
  2  |    |    |    |    |    |    |    |    |
     +----+----+----+----+----+----+----+----+
  1  | wP | wP | wP | wP | wP | wP | wP | wP |
     +----+----+----+----+----+----+----+----+
  0  | wR | wN | wB | wQ | wK | wB | wN | wR |
     +----+----+----+----+----+----+----+----+
        0    1    2    3    4    5    6    7    
Команды:
    exit                               -- выход
    move <row> <col> <row1> <col1>     -- ход из клетки (row, col)
                            

move 7 5 3 1
Ход успешен
     +----+----+----+----+----+----+----+----+
  7  | bR | bN |    | bQ | bK |    | bN | bR |
     +----+----+----+----+----+----+----+----+
  6  | bP |    | bP | bP |    | bP | bP | bP |
     +----+----+----+----+----+----+----+----+
  5  | bB | bP |    |    | bP |    |    |    |
     +----+----+----+----+----+----+----+----+
  4  |    |    |    | wP |    |    |    |    |
     +----+----+----+----+----+----+----+----+
  3  |    | bB |    |    |    |    |    |    |
     +----+----+----+----+----+----+----+----+
  2  |    |    |    |    | wP |    |    | wP |
     +----+----+----+----+----+----+----+----+
  1  | wP | wP | wP |    |    | wP | wP |    |
     +----+----+----+----+----+----+----+----+
  0  | wR | wN | wB | wQ | wK | wB | wN | wR |
     +----+----+----+----+----+----+----+----+
        0    1    2    3    4    5    6    7    
Команды:
    exit                               -- выход
    move <row> <col> <row1> <col1>     -- ход из клетки (row, col)
   

In [ ]:
move 